(C) 2023-24 Manfred Schlaegl <manfred.schlaegl@jku.at>, Institute for Complex Systems, JKU Linz

SPDX-License-Identifier: BSD 3-clause "New" or "Revised" License

In [ ]:
import sys
import time
import os
import shutil
import concurrent.futures

import rvvts

## Config

In [ ]:
RV_XLEN = 64
RVV_VLEN = 512
TESTSET_NAME = "RVV_test1"

# testcases to generate
N_RESULTS = 100

# minimum coverage of testcase
MIN_COV = 10.0

# max workers (machine dependent)
MAX_WORKERS = 10
# sub iterations before status
SUBITERATIONS = 20

In [ ]:
%run config_internal.ipynb
%run config_host.ipynb

In [ ]:
# working dir
DIR = os.getcwd() + "/run/CovGuidedTestset_RV" + str(RV_XLEN) + "_" + str(TESTSET_NAME) + "_" + str(N_RESULTS) + "_" + str(MIN_COV)

# force stop flag file -> create to stop all processes
FORCE_STOP_FILE = DIR + "/FORCE_STOP_FLAG"

# build config
config = dict()
config.update(config_internal.copy())
config.update(config_host.copy())
config.update(dict(
    dir = DIR,

    RefCovRunner_coverage = rvvts.RISCVOVPSIMCoverageRunner,
    #RISCVOVPSIMCover_metric = "basic", # basic, extended or mnemonic
    RISCVOVPSIMCover_metric = "extended", # basic, extended or mnemonic

    xlen = RV_XLEN,
    vector_vlen = RVV_VLEN,
))

## Implementation

In [ ]:
def is_force_stop():
    if os.path.isfile(FORCE_STOP_FILE):
        return True
    return False

def set_force_stop(set):
    if is_force_stop():
        os.remove(FORCE_STOP_FILE)
    if set:
        open(FORCE_STOP_FILE, "w")

In [ ]:
def generator(id):

    # don't begin when stop is requested
    if is_force_stop():
        return

    id_str = "(" + str(id) + ")"
    print("BEGIN" + id_str)

    id_dir = config["dir"] + "/" + str(id)
    exc_file = id_dir + "/EXCEPTION.log"

    try:
        # cleanup exception file
        if os.path.isfile(exc_file):
            os.remove(exc_file)

        wconfig = config.copy()
        wconfig["dir"] = id_dir
        runner = rvvts.CovGuidedFuzzerGenRunner(wconfig)

        while True:
            if is_force_stop():
                print("FORCE STOP" + id_str)
                break

            ret = runner.run(
                subiterations = SUBITERATIONS,
                min_start_fragments = 1,
                max_start_fragments = 5,
                timeout = 5.0,
                blocking = True
            )
            if ret[0] != rvvts.RunnerOutcome.COMPLETE:
                print("ERROR" + id_str)
                break
     
            # run until coverage is reached
            if ret[1][2] >= MIN_COV:
                print("MIN_COV REACHED" + id_str)
                break

    except Exception as e:
        print("EXCEPTION" + id_str)
        import traceback # SIC! (does not work if moved to header)
        with open(exc_file, "w") as file:
            traceback.print_exc(file = file)

    print("END" + id_str)

## RUN

In [ ]:
print("START " + DIR)
set_force_stop(False)
ids = []
for id in range(N_RESULTS):
    ids.append(str(id))
executor = concurrent.futures.ProcessPoolExecutor(max_workers = MAX_WORKERS)
executor.map(generator, ids)

In [ ]:
while True:
    time.sleep(1000000)

# STOP
(stop notebook and execute following line)

In [ ]:
set_force_stop(True)